## 3D Reconstruction of 2D Slices of DICOM Images

In [9]:
import os
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict

root_dir_string = "/Users/lukeyun/SDS323/manifest-1599750808610/Pancreas-CT" # Change as necessary
root_dir_list = sorted(os.listdir(root_dir_string))
data = defaultdict(list)

folder_volumes = []

# Go through the files in root directory
for folder in root_dir_list[:3]: 
    # leave root_dir_list[:3] when no need to check and just need the finished array
    # change to root_dir_list[:3] to only go through the first 3 folders which is needed when plotting 3D reconstruction for check
    
    # Skip at non-data filesp
    if not folder.startswith("PANCREAS"):
        continue
    
    # all the dicom slices for the folder
    dicom_slices_folder =[]

    # Find data files
    # Assuming all data is organized the same
    data_dir_string = list(os.walk(os.path.join(root_dir_string, folder)))[2][0]
    dcm_files = sorted(list(os.walk(os.path.join(root_dir_string, folder)))[2][2])

    # Read the data
    for image_file in dcm_files:

        cur_data = pydicom.dcmread(os.path.join(data_dir_string, image_file))
        cur_data_pixels = cur_data.pixel_array # extract only the picture
        dicom_slices_folder.append(cur_data)
        data[folder].append(cur_data_pixels) 

    #### start of 3D volume reconstruction
    # Extracting necessary information from the DICOM metadata
    spacing_between_slices = np.abs(dicom_slices_folder[0].ImagePositionPatient[2] - dicom_slices_folder[1].ImagePositionPatient[2])
    rows = dicom_slices_folder[0].Rows
    cols = dicom_slices_folder[0].Columns
    num_slices = len(dicom_slices_folder)

    # Creating an empty 3D array to store the reconstructed volume
    volume = np.zeros((rows, cols, num_slices))

    # Filling the 3D array with DICOM pixel data
    for i, dicom_slice in enumerate(dicom_slices_folder):
        volume[:, :, i] = dicom_slice.pixel_array

    folder_volumes.append((os.path.join(root_dir_string, folder), volume, spacing_between_slices))
    
    print("folder:", folder, "|number of images:", len(dcm_files), "|processed!", sep="", end="\n\n")

### Checking that the 3D reconstruction happened currently
### Only run the following code on a small number of folders
for folder_name, volume, spacing in folder_volumes:
    print("Folder: ", folder_name)
    print("Volume Shape: ", volume.shape)
    print("Spacing: ", spacing)

# visualization of 3D reconstruction with matplotlib
    # Plot the 3D volume
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Plot voxels
    x, y, z = np.indices(volume.shape) # three mesh grids that correspond to dimensions of 3D volume array
    ax.scatter(x, y, z, c=volume.flatten(), cmap='gray') # scatter plot
    # volume.flatten() determines the color of each point by its corresponding voxel intensity in the volume array

    plt.title(folder_name)
    plt.show()



folder:PANCREAS_0001|number of images:240|processed!

Folder:  /Users/lukeyun/SDS323/manifest-1599750808610/Pancreas-CT/PANCREAS_0001
Volume Shape:  (512, 512, 240)
Spacing:  1.0
